# Decoding Cellphone Signals with a Quantum Computer

This notebook demonstrates the use of a quantum computer for decoding transmissions 
in cellphone networks and compares the performance to state-of-the-art classical 
alternatives. It has the following main sections:

1. [Modeling a Network](#Modeling-a-Network) instantiates a model wireless/cellular network.
2. [Decoding Transmissions: Classical Resources](#Decoding-Transmissions:-Classical-Resources) 
    demonstrates the capabilities of standard decoding algorithms. 
3. [Decoding Transmissions: Quantum Resources](#Decoding-Transmissions:-Quantum-Resources) 
    uses a quantum computer to decode transmissions.
4. [Supplementary Technical Material](#Supplementary-Technical-Material) provides 
    additional, more-detailed technical information.

## A Hard (and Expensive) Problem

The number of base stations required to serve a large city depends on the stations' 
signal-processing capacities. Unfortunately, small increases to environmental noise 
and cellphone numbers quickly harden[[1]](#1) the problem of decoding transmissions. 
The graphic below shows the dependency of transmission errors on 
[SNRb](https://en.wikipedia.org/wiki/Signal-to-noise_ratio), signal-to-noise ratio, and 
$\frac{Tx}{Rx}$, the ratio of transmitters (e.g. cellphones) to receivers (e.g., base stations)
in wireless networks. 

<img src="_static/problem_hardness_3d.png" width="500">

The world's ubiquitous cellphones require huge numbers of base stations to provide 
its cellular networks; for example, in mid 2022, China's Ministry of Industry and Information 
Technology reported having deployed almost 
[2 million base stations](https://techblog.comsoc.org/2022/08/19/china-miit-claim-475m-5g-mobile-users-1-97m-5g-base-stations-at-end-of-july-2022/) 
for its 5G network alone.

A paper, [[2]](#2), presented in [SIGCOMM](https://www.sigcomm.org/) 2019 states, "For optimal 
performance, an amount of computation that increases at an exponential rate both with the 
number of users and with the data rate of each user is often required. The base station’s 
computational capacity is thus becoming one of the key limiting factors on wireless capacity." 

However, power consumption scales steeply with computation capacity (with the notable 
exception of quantum computers: an Advantage&#8482; requires just 25kW for its operation) and 
powering base stations has become 
[enormously challenging](https://techblog.comsoc.org/2020/08/07/5g-base-station-deployments-open-ran-competition-huge-5g-bs-power-problem/).  

Profitability for network providers is now driven by seemingly conflicting imperatives: 
(1) reducing power consumption and (2) increasing processing capability.

## What is Coordinated Multipoint?

Shortly after the first generation of cellular networks were deployed it became clear that 
the naive model of pairing cellphones to a single nearby base station is inadequate. Many 
technologies have been developed to boost performance. 
[Cooperative MIMO](https://en.wikipedia.org/wiki/Cooperative_MIMO) (multiple-input multiple-output) 
effectively exploits the spatial domain of mobile fading channels to significantly improve 
performance. In one of its variations, coordinated multipoint (CoMP), neighboring base stations 
share data and channel state information to enable them to coordinate downlink transmissions 
and jointly process received signals.

CoMP shifts processing from bases stations to a shared data center. This notebook
explores the advantages of incorporating a quantum computer into such data centers for 
improving performance with reduced power consumption.

## References

<a name="2">[2]</a> Toshiyuki Tanaka. 
A Statistical-Mechanics Approach to Large-System Analysis of CDMA Multiuser Detectors.
IEEE TRANSACTIONS ON INFORMATION THEORY, VOL. 48, NO. 11, NOVEMBER 2002

<a name="1">[1]</a> Minsung Kim, Davide Venturelli, and Kyle Jamieson. 
Leveraging quantum annealing for large MIMO processing in centralized radio access networks.
SIGCOMM '19: Proceedings of the ACM Special Interest Group on Data Communication, August 2019, Pages 241–255 
https://dl.acm.org/doi/10.1145/3341302.3342072

# Modeling a Network

A cellular network can be modelled as a lattice, with some nodes representing the cellphones 
(transmitters, denoted here as $T_x$), others the base stations (receivers, $R_x$), and dges between
$T_x$ and $R_x$ nodes model the transmission channels. In CoMP, edges between a cellphone and more 
than one base station are relevant: base stations can forward received signals, which have propagated 
through multiple channels with various noise, to be jointly processed. This aggregated processing 
reduces the bit error rate (BER) compared to decoding the cellphone's transmission over a single channel
but requires more powerful computation. 

Increasing the number of $T_x$ nodes relative to $R_x$ nodes (the $\frac{Tx}{Rx}$ ratio), raises 
the processing load, simulating base stations in densely populated areas with with many cellphones 
communicating simultaneously. Transmissions are simulated as random bit strings with amplitudes 
distorted by some random amount representing noise. 

This first code cell imports the functionality needed for the following sections. 

In [ ]:
# General Python packages
import matplotlib.pyplot as plt
import numpy as np

# Ocean software packages
import dimod
from dwave.system import DWaveSampler, FixedEmbeddingComposite

# Functions of this repository
from helpers.draw import draw_loop_comparison, draw_network
from helpers.filters import apply_filter, apply_filters, compare_signals, create_filter, create_filters, time_filter_instantiation
from helpers.general import loop_comparisons
from helpers.network import configure_network, create_channels, print_network_stats, simulate_signals

# Enable graphics inline
%matplotlib inline 

# Remove once dimod's mimo functions are updated
import warnings
warnings.filterwarnings('ignore')

## Create a Network Graph

The `configure_network` function in the code cell below returns a network based on a lattice 
that connects each node to its four nearest neighbors. $T_x$ and $R_x$ nodes alternate: 
most<sup>[1]</sup> cellphones can transmit to 4 base stations and base stations can receive signals from 4 
cellphones. 

By default, the `configure_network` function returns a network based on a $45x45$ lattice with 
a ratio of $\frac{Tx}{Rx} \approx 1.5$, achieved by randomly "turning off" some number of base stations. 
For visual clarity, this first section starts with a network based on a 12x12 lattice by setting 
`network_size=5` (for $n=5$, lattice length is given by $3(n-1)=12$ ).

<sup>[1]<sub>With the exception of nodes on the lattice boundaries.</sub></sup>

The `draw_network` function colors transmitters in red and receivers in green. 

In [ ]:
network, _ = configure_network(network_size=5)

print_network_stats(network)
draw_network(network)

## Create Channels

Next, simulate the channels between transmitters and receivers. 

By default, the `create_channels` function below returns channels generated from a binary, 
real distribution that represents 
[inter-symbol interference (ISI)](https://en.wikipedia.org/wiki/Intersymbol_interference), meaning that 
in the signal a base station receives, each element of the transmission sequence might be constituted of the 
different trqnsmitted symbols of more than one cellphone.

In [ ]:
channels, channel_power = create_channels(network)
print(f"Channels are represented by a {channels.shape[0]}x{channels.shape[1]} matrix.")

# Decoding Transmissions: Classical Resources

To distinguish between the transmitted signals from multiple cellphones, and from noise injected 
into the transmissions by the environment, base stations standardly process (decode) received 
transmissions with one of the following linear filters:

* [Matched filter](https://en.wikipedia.org/wiki/Matched_filter) is the optimal linear filter 
    for maximizing signal-to-noise ratio (SNR) of channels modelled as having additive stochastic 
    noise. It uses the technique of [convolution](https://en.wikipedia.org/wiki/Convolution) with a 
    conjugated time-reversed signal template.
* [Zero forcing](https://en.wikipedia.org/wiki/Zero-forcing_precoding) applies the inverse of the 
    channel frequency response to the received signal to restore the pre-transmission signal.
* [Minimum mean squared error (MMSE)](https://en.wikipedia.org/wiki/Minimum_mean_square_error) 
    minimizes the mean square error (MSE) between the transmitted and the received signal to 
    mitigate effects of ISI and noise. 

## Create Filters

The `create_filters` function below instantiates all three linear filters described above.

<div class="alert alert-warning" role="alert" style="margin: 10px">Note: Linear filters 
are instantiated for the channels they are intended to decode and such are short-lived. 
The importance of this is demonstrated in the following sections.</div>

In [ ]:
filters = create_filters(channels)
print(f"Created filters: {list(filters.keys())}.")

## Simulate Transmissions

Cellular and wireless networks standardly use quadrature amplitude modulation (QAM) to encode 
transmissions. 

<img src="_static/qam.png" width="600">

Typically used encodings include:

* BPSK: Binary Phase Shift Keying. Transmission symbols are the real values $+1, -1$.
* QPSK: Quadrature Phase Shift Keying. Transmission symbols are the complex values
    $1+1j, 1-1j, -1+1j, -1-1j$ (normalized by $\frac{1}{\sqrt{2}}$). 
* 16-QAM: 16 complex-valued QPSK symbols modulated by either 1 or 3.
* 64-QAM and 256-QAM Scale up 16-QAM.

By default, the `simulate_signals` function below generates BPSK signals. In this code cell, 
`transmitted_symbols` is a sequence of BPSK symbols selected from a uniform
random distribution and `y` the signal received by the base stations, the result of 
the instantiated channels acting upon the transmitted signal. 

In [ ]:
y, transmitted_symbols = simulate_signals(channels, channel_power)                 
print(f"First 10 transmitted symbols: {transmitted_symbols.flatten()[:10]}. \nFirst 10 received symbols: {y.flatten()[:10]}.")

## Decode Received Signals

Use the `apply_filters` function below to decode the signal received by the network's base stations
with each of the three linear filters. The `compare_signals` function then performs a bit-by-bit 
comparison between the received signal and the symbols transmitted by the network's cellphones.

In [ ]:
v = apply_filters(y, filters)
compare_signals(v, transmitted_symbols)

The operations of this section do **not** account for two significant factors of real-world networks:

1. Cellular networks impose strict constraints on processing time.
2. In addition to the ISI represented by the simulated channel above, wireless environments are noisy. 
   The `simulate_signals` function can represent real wireless channels by adding random normal noise 
   to the transmitted symbols.

The following sections take these factors into consideration.

# Scaling Up

In real-world networks, processing time is crucial: base stations or shared data centers must 
decode any received transmission frames (a defined number of sequential transmitted symbols) 
and return acknowledgements within a prescribed amount of time. For Wi-Fi networks, 
this imposes a hard requirement on processing time of tens of microseconds. In newer cellphone 
networks such as 4G LTE, receivers must process transmissions with similar speed due to 
redundancy schemes. 

Consequently, networks adopt processing algorithms such as linear filters that meet the speed 
requirements, even if performance is suboptimal.

As noted above, linear filters are adapted specifically for the instantiated channels; in network
terms, the filter's parameters are calibrated to the currently existing channels as periodically 
measured by having the network transmit a known pilot signal. The lifetime of such channels is 
tens of milliseconds. As a result, networks use filters that can be instantiated/calibrated fast. 

The `time_filter_instantiation` function below measures instantiation times for each of the three
linear filters for a few different sizes of networks. Any instantiation times greater than half a 
second (500 milliseconds) are highlighted in red in the output. 

A more detailed comparison is given in the [Technical Supplement](#Technical-Supplement) section below.

In [ ]:
time_filter_instantiation(network_sizes=[5, 10, 15])

The matched-filter algorithm scales up well and can be used in large wireless networks.
The other two filters require a time-consuming mathematical operation, matrix inversion. 
Matrix inversion is slow even on GPUs and does not parallelize well. 

In the following sections, which demonstrate operations on networks of size 16, only the 
matched filter algorithm is used.

# Decoding Transmissions: Quantum Resources

This section demonstrates using a quantum computer to decode wireless transmissions. 

First, select a quantum computer using [Ocean](https://docs.ocean.dwavesys.com/en/stable/index.html) 
software's [DWaveSampler](https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/samplers.html)
class.

By default, the quantum processing unit (QPU) with the greatest number of available qubits is selected.
As of August 2023, that quantum computer is bound to be an Advantage, which is based on the 
[Pegasus topology](https://docs.dwavesys.com/docs/latest/c_gs_4.html). Although unnecessary, the 
preferred topology is explicitly called as a parameter below to highlight that the code used in 
the following sections was adapted for it. For further information, see the 
[Technical Supplement](#Technical-Supplement) section below.


In [ ]:
qpu = DWaveSampler(solver=dict(topology__type="pegasus"))

print(f"Selected {qpu.solver.name} with {len(qpu.nodelist)} qubits.")

## BQM Representation (Small-Town Problem)

Any problem to be solved on a quantum computer 
[must be expressed](https://docs.dwavesys.com/docs/latest/c_gs_workflow.html) as a 
binary quadratic model (BQM). 

This section demonstrates the following operations: 

* Create a network model that can be represented on the selected QPU.
* Simulate a **noisy** transmission and decode it using the matched-filter algorithm.
* Create a BQM representing the decoding problem as solve it on the selected quantum computer.

### Create a Network, Filter, and Transmission

First, repeat the operations already done in the previous section but now using parameters 
`qpu` and `SNRb` as follows:

* `SNRb` sets the level of noise the `simulate_signals` function adds to the received signal, `y`.
* `qpu` gives the `configure_network` function access to the selected QPU's
    [working graph](https://docs.dwavesys.com/docs/latest/c_gs_4.html#the-working-graph). The function
    uses it to generate the returned `embedding` variable, which is needed in the 
    [Decode Received Signals](#Decode-Received-Signals) subsection below. 

For comparison, also decode the transmission with a matched filter. 

In [ ]:
SNRb = 10

# Create a model network
network, embedding = configure_network(network_size=16, qpu=qpu)
print_network_stats(network)

# Model the channel used for a particular transmission interval
channels, channel_power =  create_channels(network)

# Instantiate a matched filter for the transmission interval
filter_mf = create_filter(channels, method='matched_filter')
y, transmitted_symbols = simulate_signals(channels, channel_power, SNRb=SNRb)  

# Decode the transmission with the linear filter and compare to the original transmitted symbols
v = apply_filter(filter_mf, y) 
compare_signals(v, transmitted_symbols)

### Create a BQM

Ocean's [dimod](https://docs.ocean.dwavesys.com/en/stable/docs_dimod/sdk_index.html) package
provides [generators](https://docs.ocean.dwavesys.com/en/stable/docs_dimod/reference/generators.html) 
for various common problems. Below, use the `spin_encoded_comp` generator to translate the network 
problem into a BQM.

In [ ]:
bqm = dimod.generators.mimo.spin_encoded_comp(network, 
                                              modulation='BPSK',
                                              F_distribution=('binary','real'), 
                                              F=channels,
                                              y=y)

print(f"BQM has {len(bqm)} variables with {len(bqm.quadratic)} quadratic interactions.")

### Decode Received Signals

Use the selected quantum computer to decode the transmission.

Ocean software's 
[FixedEmbeddingComposite](https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/composites.html#fixedembeddingcomposite)
handles the mapping between the problem's variables, $T_x$ nodes of the network, and the QPU's
qubits, using the `embedding` variable found above as the 
[minor-embedding](https://docs.dwavesys.com/docs/latest/c_gs_7.html). 

more information about the minor-embedding and the QPU's parameters is given in the 
[Technical Supplement](#Technical-Supplement) section below. 

In [ ]:
sampler = FixedEmbeddingComposite(qpu, embedding)

sampleset = sampler.sample(bqm, 
                           num_reads=30, 
                           annealing_time=200, 
                           chain_strength=-0.13*min(bqm.linear.values()),
                           label='Notebook - Coordinated Multipoint')

compare_signals(sampleset, transmitted_symbols)

## Big-City Problems

Problems become harder[[2]](#2) as a function of channel noise and base-station load. 
The graphic below shows the bit error rate (BER) as a function of SNRb and $\frac{Tx}{Rx}$ 
for a large network (`network_size=16`). Each data point is the median of ten problems 
solved by running simulated annealing for hundreds of times longer than the network-driven 
runtimes used in this notebook (around 10 seconds versus 30 milliseconds used to demonstrate 
decoding in real wireless networks).

<img src="_static/problem_hardness.png" width="800">

One sees, for example, that for SNRb of 5 and $\frac{Tx}{Rx}=1.5$, the expected BER approaches 
10%. For problems with parameters above these BER lines, the BQM energy can be lowest (or the 
maximum likelihood greatest) for solutions that do **not** match the transmission.   

By default, the `loop_comparisons` function below sets `network_size=16`, SNRb to 5 and the $\frac{Tx}{Rx}$ 
ratio to 1.5. The next cell runs `runs` number of problems. 

In [ ]:
results = loop_comparisons(runs=10, qpu=qpu)
medians = {filter: np.median(val) for filter, val in results.items()}

for solver in medians:
    print(f'\t * Median success rate for {solver}: {medians[solver]}.')

draw_loop_comparison(results)

### Rough-Neighborhood Problems

For completeness, this subsection demonstrates a particularly hard problem by setting very high 
values of noise and load. You can configure and run for your own preferred values of `SNR` and 
`tx_to_rx` below.

In [ ]:
SNRb = 4.5
tx_to_rx = 1.65


results = loop_comparisons(runs=5, qpu=qpu, snr=SNRb, ratio=tx_to_rx)
medians = {filter: np.median(val) for filter, val in results.items()}

for solver in medians:
    print(f'\t * Median success rate for {solver}: {medians[solver]}.')

draw_loop_comparison(results, SNRb=SNRb, ratio=tx_to_rx)

# Technical Supplement

This section provides more technical details on some of the topics demonstrated above.

## Performance of Matrix-Inverting Filters 

Processing time in real-world networks is crucial: base stations or shared data centers must 
decode transmissions in tens of microseconds. 

As demonstrated above, the matched-filter algorithm scales up well and can be used in large wireless networks.
The other two filters require a time-consuming mathematical operation, matrix inversion, which is slow even on 
GPUs and does not parallelize well. 

For a range of network sizes, the following plot shows:

* Decoding success rate (left). Each data point is the median of ten problems with SNRb of 5 and $\frac{Tx}{Rx}=1.5$. 
* Instantiation time (right). Times were measure on a 10-core i5 Intel processor operating at 1600 Mhz.  

<img src="_static/all_matched_filters_performance.png" width="800">

Instantiation times that exceed a second, as they do for the matrix-inverting filters, preclude their usage
in processing above certain scales. 

## Comparison to Simulated Annealing

[Simulated annealing](https://en.wikipedia.org/wiki/Simulated_annealing) is a powerful heuristic
that dominates the space of general solvers. For many problems it is considered the benchmark 
for performance.

Like the matrix inversion required to instantiate the zero-forcing and MMSE linear filters, 
simulated annealing is computationally demanding: it requires a strong processor with high 
power consumption. 

For the comparisons of this subsection, assume that an algorithm running on a CPU requires 
10e-8 seconds per variable update: a network of size 16 has about 2000 variables ($T_x$ nodes),
so each sweep of simulated annealing takes around 200 microseconds. To compare performance to the QPU with 
its access time of about 30 milliseconds, the `loop_comparisons` function runs simulated annealing 
for 150 sweeps. By default, it runs on a network of size 16 with SNRb  of 5 and  $\frac{Tx}{Rx}=1.5$.

In [ ]:
results = loop_comparisons(runs=5, qpu=qpu, solvers=['SA'])
medians = {filter: np.median(val) for filter, val in results.items()}

for solver in medians:
    print(f'\t * Median success rate for {solver}: {medians[solver]}.')

draw_loop_comparison(results)

## Comparison to Other Classical Algorithms

For completeness, this subsection demonstrates comparisons to additional algorithms standardly used
to solve hard problems:

* [Steepest Descent](https://docs.ocean.dwavesys.com/en/stable/docs_samplers/index.html#steepest-descent) 
    a discrete analogue of the [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent) technique.
* [Tabu](https://docs.ocean.dwavesys.com/en/stable/docs_samplers/index.html#tabu) a heuristic that employs 
    local search with methods to escape local minima.

Like simulated annealing, both are computationally demanding.  

In [ ]:
results = loop_comparisons(runs=5, qpu=qpu, solvers=['greedy', 'tabu'])
medians = {filter: np.median(val) for filter, val in results.items()}

for solver in medians:
    print(f'\t * Median success rate for {solver}: {medians[solver]}.')

draw_loop_comparison(results)

### Preprocessing Performance

For completeness, this subsection also looks at the performance of 
[preprocessing](https://en.wikipedia.org/wiki/Data_Preprocessing). For some classes of problems,
a short period of preprocessing prior to running a solver can result in improved solutions and 
runtimes. 

The plot below shows the use of 
[roof duality](https://docs.ocean.dwavesys.com/en/stable/docs_preprocessing/reference/composites.html#fix-variables-composite)
to fix variables before applying a tabu sampler. Each point on the plot represents the median of 
20 problems. 

<img src="_static/preprocessing.png" width="700">

One sees that as problems harden---size and $\frac{T_x}{R_x}$ increase and SNRb decreases---preprocessing 
loses effectiveness.

## Configuring Optimal QPU Parameters

Ocean software selects reasonable default parameters when submitting problems to the 
quantum computer. However, when performance in crucial, as in these problems, where 
differences of a percent matter, it is important to configure the QPU with optimal 
parameter values. D-Wave's [system documentation](https://docs.dwavesys.com/docs/latest/doc_handbook.html)  
provides guidance on optimizing QPU use.

Here, two non-default parameters are used:

* [annealing_time](https://docs.dwavesys.com/docs/latest/c_solver_parameters.html#annealing-time) sets 
    the duration, in microseconds, of quantum annealing time, per read. 

    For time-critical problems, rather than use the default 20 microseconds with a large 
    number of reads, a good rule of thumb is to balance the programming and sampling times 
    in the overall [QPU access time](https://docs.dwavesys.com/docs/latest/c_qpu_timing.html#qpu-access-time).
    Using the QPU access time 
    [estimation](https://docs.dwavesys.com/docs/latest/c_qpu_timing.html#estimating-access-time) algorithm 
    and tool provided in Ocean produces an optimal annealing time of just over 200 microseconds for 
    most the problems in this example.
     
* [Chain strength](https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/generated/dwave.system.composites.FixedEmbeddingComposite.sample.html)
    sets the coupling strength between qubits that form a 
    [chain](https://docs.dwavesys.com/docs/latest/c_gs_7.html#chains). 

    For most problems, the default 
    [chain-strength calculation](https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/generated/dwave.embedding.chain_strength.uniform_torque_compensation.html)
    produces decent results. For this problem, however, it does not.

    The following plot shows the bit error rate (BER) and the fraction of broken chains for a range
    of chain strengths, calculated as `-cs_multiplier*min(bqm.linear.values())`, where `cs_multiplier` 
    is a multiplier of the most negative value of $h_i$, the linear coefficients of the BQM. Each data 
    point is the median of twenty problems with SNRb of 5 and $\frac{Tx}{Rx}=1.5$ for a large network (`network_size=16`). 

    An optimal chain strength for the network problems is around $-0.13\min{h_i}$.

<img src="_static/chain_strength.png" width="700">

## Minor Embedding

For most problems submitted directly to the QPU, mapping between the problem's variables and 
QPU qubits is heuristic, automated by such tools as Ocean's 
[minorminer](https://docs.ocean.dwavesys.com/en/stable/docs_minorminer/source/sdk_index.html) 
package. 

Although that package also provides algorithms for minor-embedding graphs with regularities, 
one can produce more efficient embeddings by tiling an optimal cell embedding across the full
graph. However, because the 
QPU [working graph](https://docs.dwavesys.com/docs/latest/c_gs_4.html#the-working-graph) 
can contain defects due to some small number of qubits and couplers being removed for failing 
to meet the specifications to function as desired, one needs to shift the mapping around those. 
The `configure_network` function handles this.

The graphic below shows how a small network graph (`network_size=4`) is minor-embedded into the 
selected QPU's working graph. 

Colored dots are qubits: note that pairs of identically-colored dots, such as the blue, green, and red pairs on the top row, represent a single node of the logical network. 

<img src="_static/embedding_lattice4.png" width="700">


## Advantage2: Boosting Performance 